<a href="https://colab.research.google.com/github/Sankytanky100/Data_Science-Projects/blob/main/Attribution_Queries_Using_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project Overview
Objective: Analyze marketing campaign performance for CoolTShirts by mapping customer journeys using touch attribution and advanced SQL queries.

Dataset: We will create a simulated page_visits table containing user journey data, including campaigns and sources.

In [ ]:
# Install ipython-sql extension
!pip install ipython-sql

# Load the SQL extension
%load_ext sql

# Create an in-memory SQLite database
%sql sqlite://


In [ ]:
%%sql

DROP TABLE IF EXISTS page_visits;

CREATE TABLE page_visits (
    user_id INTEGER,
    visit_time TIMESTAMP,
    page_name TEXT,
    campaign TEXT,
    source TEXT
);


In [ ]:
%%sql

INSERT INTO page_visits (user_id, visit_time, page_name, campaign, source) VALUES
(1, '2023-11-01 08:00:00', '1 - Home', 'Campaign A', 'Google'),
(1, '2023-11-01 08:05:00', '2 - Product', 'Campaign A', 'Google'),
(1, '2023-11-01 08:10:00', '4 - Purchase', 'Campaign A', 'Google'),

(2, '2023-11-01 09:00:00', '1 - Home', 'Campaign B', 'Facebook'),
(2, '2023-11-01 09:15:00', '3 - Cart', 'Campaign C', 'Instagram'),
(2, '2023-11-01 09:20:00', '4 - Purchase', 'Campaign C', 'Instagram'),

(3, '2023-11-01 10:00:00', '2 - Product', 'Campaign D', 'Twitter'),
(3, '2023-11-01 10:05:00', '3 - Cart', 'Campaign D', 'Twitter'),

(4, '2023-11-01 11:00:00', '1 - Home', 'Campaign E', 'LinkedIn'),
(4, '2023-11-01 11:10:00', '2 - Product', 'Campaign E', 'LinkedIn'),
(4, '2023-11-01 11:15:00', '4 - Purchase', 'Campaign E', 'LinkedIn'),

(5, '2023-11-01 12:00:00', '1 - Home', 'Campaign F', 'Google'),
(5, '2023-11-01 12:05:00', '2 - Product', 'Campaign F', 'Google'),

(6, '2023-11-01 13:00:00', '1 - Home', 'Campaign A', 'Google'),
(6, '2023-11-01 13:10:00', '4 - Purchase', 'Campaign A', 'Google');


In [ ]:
%%sql

SELECT * FROM page_visits;


In [ ]:
%%sql

SELECT COUNT(DISTINCT campaign) AS num_campaigns
FROM page_visits;

SELECT COUNT(DISTINCT source) AS num_sources
FROM page_visits;

SELECT DISTINCT campaign, source
FROM page_visits
ORDER BY campaign;

In [ ]:
%%sql

SELECT DISTINCT page_name
FROM page_visits
ORDER BY page_name;


In [ ]:
%%sql

WITH first_touch AS (
    SELECT
        user_id,
        MIN(visit_time) AS first_visit_time
    FROM page_visits
    GROUP BY user_id
)
SELECT
    pv.campaign,
    COUNT(DISTINCT pv.user_id) AS num_first_touches
FROM first_touch ft
JOIN page_visits pv ON ft.user_id = pv.user_id AND ft.first_visit_time = pv.visit_time
GROUP BY pv.campaign
ORDER BY num_first_touches DESC;


In [ ]:
%%sql

WITH last_touch AS (
    SELECT
        user_id,
        MAX(visit_time) AS last_visit_time
    FROM page_visits
    GROUP BY user_id
)
SELECT
    pv.campaign,
    COUNT(DISTINCT pv.user_id) AS num_last_touches
FROM last_touch lt
JOIN page_visits pv ON lt.user_id = pv.user_id AND lt.last_visit_time = pv.visit_time
GROUP BY pv.campaign
ORDER BY num_last_touches DESC;


In [ ]:
%%sql

SELECT COUNT(DISTINCT user_id) AS num_purchasers
FROM page_visits
WHERE page_name = '4 - Purchase';


In [ ]:
%%sql

WITH last_touch AS (
    SELECT
        user_id,
        MAX(visit_time) AS last_visit_time
    FROM page_visits
    GROUP BY user_id
)
SELECT
    pv.campaign,
    COUNT(DISTINCT pv.user_id) AS num_purchase_last_touches
FROM last_touch lt
JOIN page_visits pv ON lt.user_id = pv.user_id AND lt.last_visit_time = pv.visit_time
WHERE pv.page_name = '4 - Purchase'
GROUP BY pv.campaign
ORDER BY num_purchase_last_touches DESC;


In [ ]:
%%sql

WITH first_touch AS (
    SELECT
        user_id,
        MIN(visit_time) AS first_visit_time
    FROM page_visits
    GROUP BY user_id
),
first_touch_counts AS (
    SELECT
        pv.campaign,
        COUNT(DISTINCT pv.user_id) AS num_first_touches
    FROM first_touch ft
    JOIN page_visits pv ON ft.user_id = pv.user_id AND ft.first_visit_time = pv.visit_time
    GROUP BY pv.campaign
),
last_touch AS (
    SELECT
        user_id,
        MAX(visit_time) AS last_visit_time
    FROM page_visits
    GROUP BY user_id
),
last_touch_counts AS (
    SELECT
        pv.campaign,
        COUNT(DISTINCT pv.user_id) AS num_last_touches
    FROM last_touch lt
    JOIN page_visits pv ON lt.user_id = pv.user_id AND lt.last_visit_time = pv.visit_time
    GROUP BY pv.campaign
),
purchase_last_touch_counts AS (
    SELECT
        pv.campaign,
        COUNT(DISTINCT pv.user_id) AS num_purchase_last_touches
    FROM last_touch lt
    JOIN page_visits pv ON lt.user_id = pv.user_id AND lt.last_visit_time = pv.visit_time
    WHERE pv.page_name = '4 - Purchase'
    GROUP BY pv.campaign
)
SELECT
    ftc.campaign,
    ftc.num_first_touches,
    ltc.num_last_touches,
    plc.num_purchase_last_touches
FROM first_touch_counts ftc
LEFT JOIN last_touch_counts ltc ON ftc.campaign = ltc.campaign
LEFT JOIN purchase_last_touch_counts plc ON ftc.campaign = plc.campaign
ORDER BY plc.num_purchase_last_touches DESC NULLS LAST;
